In [1]:
import turicreate as tc

In [2]:
tc.config.set_num_gpus(0)

In [3]:
# Load the date
data =  tc.SFrame("./ig02.sframe/")

# Make a train-test split
train_data, test_data = data.random_split(0.8)

In [ ]:
# Create a model
model = tc.object_detector.create(train_data, batch_size = 512)

# Save predictions to an SArray
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)

Using 'image' as feature column
Using 'annotations' as annotations column
Setting 'batch_size' to 512
Using CPU to create model
Setting 'max_iterations' to 1000


In [ ]:
# Save the model for later use in Turi Create
model.save('mymodel.model')

# Export for use in Core ML
model.export_coreml('MyCustomObjectDetector.mlmodel')